# Lab 3 - OpenAI Agents SDK + TOOLS!!

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import gradio as gr
import os
import requests

load_dotenv(override=True)

True

In [2]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
def send_push_notification(message: str):
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [18]:
send_push_notification("Hello, me!")

In [17]:
@function_tool
def push(message: str) -> str:
    """Send a text message as a push notification to Salma with this brief message

    Args:
        message: The short text message to push to Salma
    """

    send_push_notification(message)
    return "Push notification sent"

In [12]:
push

FunctionTool(name='push', description='Send a text message as a push notification to Salma with this brief message', params_json_schema={'properties': {'message': {'description': 'The short text message to push to Salma', 'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000018D929D5260>, strict_json_schema=True, is_enabled=True)

In [13]:
instructions = """
You represent the AI Digital Twin of a human called Salma Wahwah.
You are friendly and amiable, and you introduce yourself as Salma's Digital Twin.
Salma is the senior software engineer
If you don't know the answer, send a push notification to Salma to tell her the question you couldn't answer, so that she adds it to the knowledge base.
"""

In [14]:
agent = Agent(name="Twin", instructions=instructions, model="gpt-4.1-mini", tools=[push])

In [15]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = Runner.run_streamed(agent, messages)
    reply = ""
    async for event in response.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            reply += event.data.delta
            yield reply

In [16]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
